https://divamgupta.com/image-segmentation/2019/06/06/deep-learning-semantic-segmentation-keras.html Refer to this for reasource


TODO: Add optimisers

In [ ]:
!pip install keras-segmentation
#https://github.com/divamgupta/image-segmentation-keras


In [60]:
#make rgb images and segmentation images share a file name
#get image in the right format

import os
from numpy import asarray
from PIL import Image

rgb_img = r'/home/test_images'
seg_img = r'/home/test_segmentation/'
rename = "test_"
i = 0

def remove_chars(filename):
  new_filename = ""
  for i in range(0, len(filename)):
    char = filename[i]
    if char.isnumeric():
      new_filename += char
  return new_filename + ".png"

for filename in os.listdir(rgb_img):
    old_path = os.path.join(rgb_img, filename)
    if not os.path.isdir(old_path):
      i += 1
      new_filename = remove_chars(filename)
      os.rename(os.path.join(rgb_img, filename), os.path.join(rgb_img, new_filename))

i = 0
for filename in os.listdir(seg_img):
    old_path = os.path.join(seg_img, filename)
    if not os.path.isdir(old_path):
      i += 1
      new_filename = remove_chars(filename)
      new_name = os.path.join(seg_img,new_filename)
      os.rename(os.path.join(seg_img, filename), os.path.join(seg_img, new_name))
      
      
      #Normalize image to fit model system: https://machinelearningmastery.com/how-to-manually-scale-image-pixel-data-for-deep-learning/ 
      image = Image.open(new_name)
      pixels = asarray(image)
      # convert from integers to floats
      pixels = pixels.astype('float32')
      # normalize to the range 0-1 then to 9
      pixels /= 255.0
      pixels *= 9.0
      print('Min: %.3f, Max: %.3f' % (pixels.min(), pixels.max()))
      im = Image.fromarray(pixels.astype(np.uint8))
      im.save() 

Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000, Max: 9.000
Min: 0.000,

In [61]:
!python -m keras_segmentation verify_dataset \
 --images_path="/home/test_images" \
 --segs_path="/home/test_segmentation/"  \
 --n_classes=10

100% 60/60 [00:01<00:00, 38.92it/s]
Dataset verified! 


In [18]:
import cv2
import numpy as np
from keras_segmentation import *

Create numpy dataset for data

In [68]:
from keras.models import Model, model_from_json
from keras.layers import Input, Dense, Conv2D, Dropout, MaxPooling2D, concatenate, UpSampling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
n_classes = 10
input_height = 1080
input_width = 720
img_input = Input(shape=(input_height,input_width , 3 ))

conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(img_input)
conv1 = Dropout(0.2)(conv1)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D((2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Dropout(0.2)(conv2)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D((2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Dropout(0.2)(conv3)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

up1 = concatenate([UpSampling2D((2, 2))(conv3), conv2], axis=-1)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
conv4 = Dropout(0.2)(conv4)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)

up2 = concatenate([UpSampling2D((2, 2))(conv4), conv1], axis=-1)
conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
conv5 = Dropout(0.2)(conv5)
conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)

out = Conv2D( n_classes, (1, 1) , padding='same')(conv5)

from keras_segmentation.models.model_utils import get_segmentation_model

model = get_segmentation_model(img_input ,  out,  optimizer=opt ) # this would build the segmentation model

ImportError: ignored

In [62]:
model.train( 
    train_images =  "/home/test_images/",
    train_annotations = "/home/test_segmentation/",
    checkpoints_path = "/home/weights/" , epochs=5
)

Verifying training dataset


100%|██████████| 60/60 [00:01<00:00, 37.57it/s]
/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Dataset verified! 
Epoch 1/5
167/512 [========>.....................] - ETA: 3:13:34 - loss: 6.7938 - accuracy: 0.1023

KeyboardInterrupt: ignored